In [1]:
from scipy.signal import hilbert
from scipy.signal import welch
from scipy import signal
#import METHODS_chaos
import pandas as pd
from mne.time_frequency import psd_array_multitaper, psd_array_welch
from fooof import FOOOF
import numpy as np
import argparse
import mne.io
import mne
import os
import multiprocessing as mp
from scipy.io import loadmat
from scipy.io import savemat

import glob

In [ ]:
out_dir = 'data/output/Slope/'
in_dir = 'data/input/10s/'

lfreq = 1
hfreq = 45

# output
Slope = []
Slope_space = []
sub_all = []
day_all = []   
condition_all = []

# make output directory
if not os.path.exists(out_dir):
    os.makedirs(out_dir)

# load patient info and conditions
paths = glob.glob(f'{in_dir}*.fif')    
paths.sort()

#loop over all conditions and particiants
for i, path in enumerate(paths):
    basename = os.path.basename(path)
    sub = basename[:4]
    day = basename[5:9]
    condition = basename[10:-8]
    if len(basename)==36:
         condition = basename[10:15]+basename[-5:-4]


    print(f"Analyzing Synchrony of {path}");

    #################################
    #          LOAD  DATA          #
    #################################

    epochs_mne = mne.read_epochs(path, preload=True)
    epochs_mne.interpolate_bads(reset_bads=True)
    epochs_mne.pick_types(eeg=True, meg=False, stim=False, eog=False, ecg=False, emg=False, misc=False, exclude='bads')

    epochs = epochs_mne.get_data()[:,:32,:]
    fs = 256 #check this


    # if data is too long only use the first 3 min of data
    nr_trials = min([len(epochs),80])  #ADJSUT, check 
    nr_channels =  epochs.shape[1]

    # search individual lowpass frequency
    fm = FOOOF()
    # Set the frequency range to fit the model
    freq_range = [lfreq, hfreq]
    data_con = np.concatenate(epochs,axis = 1)
    # get psd of channels
    freqs, psds = signal.welch(data_con,fs,nperseg=5*1024)

    # Get average Slope
    psds_mean =  np.mean(psds,axis = 0)
    fm.fit(freqs, psds_mean, freq_range)
    slope_id = -fm.aperiodic_params_[1]
    Slope.append(slope_id)

    Slope_space_id = []
    # Get Space-resolved Slope
    for ch in range(len(psds)):
        fm.fit(freqs, psds[ch,:], freq_range)
        slope_id = -fm.aperiodic_params_[1]
        Slope_space_id.append(slope_id)

    Slope_space.append(Slope_space_id)

    sub_all.append(sub)
    day_all.append(day)
    condition_all.append(condition)

    # save part
    output_df = {'sub':sub_all, 'day': day_all,'condition':condition_all,'Slope':Slope}
    output_df = pd.DataFrame(output_df)
    output_df.to_csv(f'{out_dir}/Slope.csv', index=False, sep=',')

    # save part
    output_df_space = {'sub':sub_all, 'day': day_all,'condition':condition_all}
    output_df = pd.concat((pd.DataFrame(output_df_space), pd.DataFrame(Slope_space).reset_index(drop=True)), axis = 1)
    output_df.to_csv(f'{out_dir}/Slope_space.csv', index=False, sep=',')


Analyzing Synchrony of data/input/10s/sub0-day1-jhana-epo.fif
Reading /Users/jonasmago/PhD_code_data/github/Criticality_PCI_Anesthesia/data/input/10s/sub0-day1-jhana-epo.fif ...
    Found the data of interest:
        t =       0.00 ...    9996.09 ms
        0 CTF compensation matrices available
Not setting metadata
100 matching events found
No baseline correction applied
0 projection items activated
Setting channel interpolation method to {'eeg': 'spline'}.
Interpolating bad channels.
    Automatic origin fit: head of radius 95.7 mm
Computing interpolation matrix from 30 sensor positions
Interpolating 2 sensors
NOTE: pick_types() is a legacy function. New code should use inst.pick(...).


/var/folders/bq/hl737msd54q59fzf7spf0r7h0000gn/T/ipykernel_88978/938256765.py:37: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  epochs = epochs_mne.get_data()[:,:32,:]


Analyzing Synchrony of data/input/10s/sub0-day1-mindfulness-epo.fif
Reading /Users/jonasmago/PhD_code_data/github/Criticality_PCI_Anesthesia/data/input/10s/sub0-day1-mindfulness-epo.fif ...
    Found the data of interest:
        t =       0.00 ...    9996.09 ms
        0 CTF compensation matrices available
Not setting metadata
120 matching events found
No baseline correction applied
0 projection items activated
Setting channel interpolation method to {'eeg': 'spline'}.
NOTE: pick_types() is a legacy function. New code should use inst.pick(...).


/var/folders/bq/hl737msd54q59fzf7spf0r7h0000gn/T/ipykernel_88978/938256765.py:34: RuntimeWarning: No bad channels to interpolate. Doing nothing...
  epochs_mne.interpolate_bads(reset_bads=True)
/var/folders/bq/hl737msd54q59fzf7spf0r7h0000gn/T/ipykernel_88978/938256765.py:37: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  epochs = epochs_mne.get_data()[:,:32,:]


Analyzing Synchrony of data/input/10s/sub0-day2-jhana-epo.fif
Reading /Users/jonasmago/PhD_code_data/github/Criticality_PCI_Anesthesia/data/input/10s/sub0-day2-jhana-epo.fif ...
    Found the data of interest:
        t =       0.00 ...    9996.09 ms
        0 CTF compensation matrices available
Not setting metadata
106 matching events found
No baseline correction applied
0 projection items activated
Setting channel interpolation method to {'eeg': 'spline'}.
Interpolating bad channels.
    Automatic origin fit: head of radius 95.7 mm
Computing interpolation matrix from 29 sensor positions
Interpolating 3 sensors
NOTE: pick_types() is a legacy function. New code should use inst.pick(...).


...

/var/folders/bq/hl737msd54q59fzf7spf0r7h0000gn/T/ipykernel_88978/938256765.py:37: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  epochs = epochs_mne.get_data()[:,:32,:]


Analyzing Synchrony of data/input/10s/sub9-day4-jhana-epo.fif
Reading /Users/jonasmago/PhD_code_data/github/Criticality_PCI_Anesthesia/data/input/10s/sub9-day4-jhana-epo.fif ...
    Found the data of interest:
        t =       0.00 ...    9996.09 ms
        0 CTF compensation matrices available
Not setting metadata
69 matching events found
No baseline correction applied
0 projection items activated
Setting channel interpolation method to {'eeg': 'spline'}.
Interpolating bad channels.
    Automatic origin fit: head of radius 95.7 mm
Computing interpolation matrix from 29 sensor positions
Interpolating 3 sensors
NOTE: pick_types() is a legacy function. New code should use inst.pick(...).


/var/folders/bq/hl737msd54q59fzf7spf0r7h0000gn/T/ipykernel_88978/938256765.py:37: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  epochs = epochs_mne.get_data()[:,:32,:]


Analyzing Synchrony of data/input/10s/sub9-day4-mindfulness-epo.fif
Reading /Users/jonasmago/PhD_code_data/github/Criticality_PCI_Anesthesia/data/input/10s/sub9-day4-mindfulness-epo.fif ...
    Found the data of interest:
        t =       0.00 ...    9996.09 ms
        0 CTF compensation matrices available
Not setting metadata
125 matching events found
No baseline correction applied
0 projection items activated
Setting channel interpolation method to {'eeg': 'spline'}.
Interpolating bad channels.
    Automatic origin fit: head of radius 95.7 mm
Computing interpolation matrix from 28 sensor positions
Interpolating 4 sensors
NOTE: pick_types() is a legacy function. New code should use inst.pick(...).


/var/folders/bq/hl737msd54q59fzf7spf0r7h0000gn/T/ipykernel_88978/938256765.py:37: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  epochs = epochs_mne.get_data()[:,:32,:]
